
# 🌿 PlantDocBot – Unified Training Notebook (Google Colab)

This notebook **combines all previous Colab files into ONE clean, executable pipeline**:
- Dataset loading
- Model creation
- Training with class weights
- Validation
- Model saving for Streamlit app

Run cells **top to bottom**.


In [ ]:

# =====================
# 1️⃣ Imports & Setup
# =====================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from collections import Counter
import os


In [ ]:

# =====================
# 2️⃣ Device Configuration
# =====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:

# =====================
# 3️⃣ Dataset Paths
# =====================
# Update this path if needed
DATA_DIR = "/content/dataset"

train_dir = os.path.join(DATA_DIR, "train")
val_dir = os.path.join(DATA_DIR, "val")


In [ ]:

# =====================
# 4️⃣ Image Transforms
# =====================
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [ ]:

# =====================
# 5️⃣ Load Dataset
# =====================
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
val_data = datasets.ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

num_classes = len(train_data.classes)
print("Classes:", train_data.classes)


In [ ]:

# =====================
# 6️⃣ Class Weights (Imbalance Handling)
# =====================
counts = Counter(train_data.targets)
total = sum(counts.values())

class_weights = [total / counts[i] for i in range(num_classes)]
weights = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)


In [ ]:

# =====================
# 7️⃣ Model Setup (ResNet50)
# =====================
model = models.resnet50(weights=None)
model.fc = nn.Linear(2048, num_classes)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:

# =====================
# 8️⃣ Training Loop
# =====================
EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Loss: {train_loss:.2f} | Val Acc: {acc:.2f}%")


In [ ]:

# =====================
# 9️⃣ Save Model for Streamlit App
# =====================
os.makedirs("models", exist_ok=True)

torch.save({
    "model_state_dict": model.state_dict(),
    "class_names": train_data.classes
}, "models/plant_disease_resnet50.pth")

print("✅ Model saved as models/plant_disease_resnet50.pth")



## ✅ Done!
You can now:
- Download `plant_disease_resnet50.pth`
- Place it inside your **Streamlit `models/` folder**
- Run `streamlit run app.py`
